## Load libraries

In [21]:
import os
import time
import pickle
import datetime
import warnings
from sentence_embeddings.config import *
import tensorflow as tf
from sentence_embeddings.model import layers
import nltk
import json
import itertools
from numpy import array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import numpy as np
import random
from word_embeddings.predict import Prediction
from char_embeddings.preprocess import sample
from tensorflow.keras.models import load_model
import re
from char_embeddings.preprocess import preprocess_data, generate_one_hot_encoding
import json

print("Tensorflow version: ",tf.__version__)

Tensorflow version:  2.2.0


## Load tokenizer

In [33]:
embedding = "word"
if embedding == "character":
    char_indices = {' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 
                    'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 
                    'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 
                    'y': 25, 'z': 26}
    indices_char = {0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 
                    10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 
                    19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
    chars = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 
             'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']
    print("Character indices/tokenizer: ",char_indices)
elif embedding == "word":
    tokenizer_path = '/home/smuthi2s/perl5/NLP/Image_Storyteller/word_embeddings/working/tokenizer_30000.json'
    with open(tokenizer_path) as f:
        data = json.load(f)
    tokenizer = tokenizer_from_json(data)
    N = 20    
    out = dict(itertools.islice(tokenizer.word_counts.items(), N))
    print("Dictionary limited by K is : " + str(out))
elif embedding == "sentence":
    tokenizer_path = "/home/smuthi2s/perl5/NLP/Image_Storyteller/sentence_embeddings/new_tokenizer_2000.json"
    with open(tokenizer_path) as f:
        data = json.load(f)
    tokenizer = tokenizer_from_json(data)
    N = 20    
    out = dict(itertools.islice(tokenizer.word_counts.items(), N))
    print("Dictionary limited by K is : " + str(out))



Dictionary limited by K is : {'the': 23040, 'half': 181, 'ling': 22, 'book': 129, 'one': 1274, 'in': 5185, 'fall': 56, 'of': 7967, 'igneeria': 30, 'series': 29, 'kaylee': 2, 'soderburg': 2, 'copyright': 10, '2013': 1, 'all': 1202, 'rights': 10, 'reserved': 10, 'isbn': 3, '1492913731': 1, '13': 7}


In [34]:
if embedding == "sentence":
    text = "start with"
    text_2 = "this is just great seth shouted ."
    idx2word = {v:k for k,v in tokenizer.word_index.items()}
    encoded_data = tokenizer.texts_to_sequences([text])[0]
    encoded_data_2 = tokenizer.texts_to_sequences([text_2])[0]
    max_length = 6
    total_sentences = [encoded_data,encoded_data_2]
    model = layers.skip_thoughts(thought_size=thought_size, word_size=embed_dim, vocab_size=10000,
                                 max_length=max_length)
    checkpoint_dir = "/scratch/smuthi2s/NLP_data/logs"
    checkpoint_path = "/scratch/smuthi2s/NLP_data/logs_480/200_ckpt-50"
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    lengths = np.array([len(encoded_data),len(encoded_data_2)])
    tf.train.load_checkpoint(latest)
    for i in range(10):
        # print(total_sentences[-2:])
        padded_data = pad_sequences(total_sentences[-2:],maxlen = max_length,padding='pre')
        masked_prev_pred, masked_next_pred = model(padded_data,lengths)
        total_sentences.append(list(np.argmax(masked_next_pred,axis=2)[0]))
        lengths = np.append(lengths,len(np.argmax(masked_next_pred,axis=2)[0]))
    story=''
    for idx,i in enumerate(total_sentences):
        i = np.unique(i)
        if idx>0:
            for j in i:
                if j!=0:
                    story+=idx2word[j]+" "
    print(story)
elif embedding == "word":
    max_length = 179
    pred = Prediction(tokenizer,max_length)
    checkpoint_path = "/home/smuthi2s/perl5/NLP/Image_Storyteller/word_embeddings/working/lang_model_30000.h5"
    pred.load_model(checkpoint_path)
    print(pred.predict_sequnce("A unknown man standing",100))
elif embedding == "character":
    checkpoint_path = "/home/smuthi2s/perl5/NLP/Image_Storyteller/char_embeddings/char_story_generator_20100.h5"
    model = load_model(checkpoint_path)
    variance = 0.5
    maxlen = 40
    sentence = 'a boy'
    generated = ''
    original = sentence
    window = sentence
    # Predict the next 400 characters based on the seed
    for i in range(100):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(window):
            x[0, t, char_indices[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, variance)
        next_char = indices_char[next_index]

        generated += next_char
        window = window[1:] + next_char

    print(original + generated)



A unknown man standing in the ground and the saw the only thing was a man who had been a few of a man who had been in the world and he had been in the world and he was a little of his family who had been in the world and he had been in the world and he was a little of his best friend in a world and he was a man who had been a few inches of the best in the world was a man who had been in the world and he had been in the world and
